In [2]:
from glob import glob
import os
from tqdm import tqdm

images = glob('/media/tekim/DATA/Dataset/화재 발생 예측 영상/Training/[[]원천]화재씬/*')

labels = []
for j,i in tqdm(enumerate(images)):
    if os.path.exists(i.replace('원천','라벨')[:-3] + 'json'):
        labels.append(i.replace('원천','라벨')[:-3] + 'json')

# images2 
print(len(labels))
print(len(images))

13159it [00:00, 16341.23it/s]

13159
13159


In [3]:
# con_labels= glob('/home/tekim/Downloads/fire_val/Validation/*/*.txt')
# print(len(con_labels))

In [4]:
a = set()
for i in labels:
    a.add(os.path.basename(i)[:-4] + 'jpg')
b = set()
for i in images:
    b.add(os.path.basename(i))

In [5]:
print(len(a))
print(len(b))

13159
13159


In [6]:
c = list(a-b)

In [7]:
print(len(set(labels)))
print(len(set(images)))

13159
13159


In [8]:
from multiprocessing import Process

In [9]:
import json 
import os 
from PIL import Image
import shutil 
from tqdm import tqdm 

In [10]:
os.path.dirname(i).split('씬')[0] + '씬'

'/media/tekim/DATA/Dataset/화재 발생 예측 영상/Training/[원천]화재씬'

In [11]:
# for i in tqdm(images):
#     des = os.path.dirname(i).split('씬')[0] + '씬'
#     try:
#         shutil.move(i,des)
#     except:
#         continue



In [12]:
def cvt_yoloformat(xyxy,w,h):
    box_w = xyxy[2] - xyxy[0]
    box_h = xyxy[3] - xyxy[1]
    return [(xyxy[0] + box_w/2)/w ,(xyxy[1] + box_h/2)/h, box_w/w, box_h/h ]

In [16]:
count1 = 0
count2 = 0
strange_img = []
flag = False
for i in tqdm(labels):
    if flag:
        break
    try:
        with open(i, 'r') as f:
            datas = json.load(f)
    except:
        try:
            with open(i, 'r',encoding='utf-8-sig') as f:
                datas = json.load(f)    
        except:
            count1 +=1
            continue
    # print(i)

    # img_name = datas['image']['filename']
    img_name = os.path.basename(i)[:-4] +'jpg'
    img_path = i.replace('라벨', '원천')[:-4] + 'jpg'

    if not os.path.exists(img_path):
        print(1)
        continue
    try:
        img = Image.open(img_path)
    except:
        strange_img.append(img_path)
        print(2)
        continue

    width,height = img.size
    # img_path = i.replace('라벨','원천').split('')[0] + '/' + img_name

    annos = datas['annotations']

    new_datas = []
    # try:
    for j in range(len(annos)):

        if 'polygon' in list(annos[j].keys()):
            poly = annos[j]['polygon']
            max_x, max_y, min_x, min_y = 0,0,1920,1080
            
            for k in poly:
                if len(k) <=1:
                    continue
                p0, p1 = int(k[0]) , int(k[1])
                if(p1 >= 1081):
                    print(k)
                    flag = True
                    break
                max_x = max(max_x,p0)
                max_y = max(max_y,p1)
                min_x = min(min_x,p0)
                min_y = min(min_y,p1)

            bbox = [min_x,min_y,max_x,max_y]
            print(bbox)
            xywh = cvt_yoloformat(bbox,width,height)
            if (xywh[2] > 1) | (xywh[3] > 1):
                print(bbox)
                print(xywh)
                print(i)
                print(img.size)
                print(poly)
                continue
                flag = True
                break
            
            # print(xywh)
            # new_datas.append(bbox)

            abc = [int(annos[j]['class']), *xywh]
            new_datas.append(abc)

        else:
            try:

                bbox = annos[j]['box']

                # print(bbox)
                xywh = cvt_yoloformat(bbox,width,height)
                abc = [int(annos[j]['class']), *xywh]
                new_datas.append(abc)
            except:
                print(annos[j].keys())
                count2 +=1
                continue
                
    # except:
    #     count2 +=1
    # print(i)
    lab_dir = os.path.dirname(i).replace('[', '').replace(']','_')
    # print(lab_dir)
    # break
    # print(lab_dir + os.path.basename(i)[:-4] + 'txt')
    # with open(lab_dir +'/' + os.path.basename(i)[:-4] + 'txt','w') as f:
    #     for j in new_datas:
    #         j = [str(k) for k in j]
    #         f.write(' '.join(j) + '\n')
    break
print('count1 : ',count1)
print(count2)


  0%|          | 0/13159 [00:00<?, ?it/s]

[0, 0, 1170, 960]
count1 :  0
0


In [15]:
print(count1)
print(count2)

0
0


In [139]:
a = ['1','2','3','4']
str(a)


"['1', '2', '3', '4']"

In [42]:
txts1= glob('/home/tekim/Downloads/fire_val/Validation/라벨화재씬/*.txt')
txts2= glob('/home/tekim/Downloads/fire_val/Validation/라벨무관씬/*.txt')
txts3= glob('/home/tekim/Downloads/fire_val/Validation/라벨유사씬/*.txt')

In [47]:
import random

random.seed(42)
random.shuffle(txts1)
random.seed(42)
random.shuffle(txts2)
random.seed(42)
random.shuffle(txts3)

In [49]:
import shutil

In [52]:
def sampling(txts,des_dir):
    for i in txts[:2000]:
        shutil.copy(i,des_dir + '/labels')
        tmp = i.replace('라벨','원천')[:-4] + '.jpg'
        shutil.copy(tmp,des_dir + '/images')



In [53]:
sampling(txts1 , des_dir='/home/tekim/datasets/fire')
sampling(txts2 , des_dir='/home/tekim/datasets/fire')
sampling(txts3 , des_dir='/home/tekim/datasets/fire')

In [46]:
print(len(txts3))

75039


In [27]:
a= ['1' ,'2','3']
' '.join(a[1:])

'2 3'

In [25]:
print(len(imgs1))
print(len(imgs2))
print(len(imgs3))
len(txts)

493159
548000
243351


1284212

In [ ]:
print(len(imgs1))
print(len(imgs2))
print(len(imgs3))
len(txts)

493159
548000
243351


In [19]:
len(txts)

1284212

In [32]:
len(txts)

1267820

In [38]:
len(txts)

1284212

In [21]:
imgses  = imgs1 + imgs2 + imgs3

In [22]:
len(imgses)

1284510

In [ ]:
imgs2 = glob('')

In [21]:
len(txts)

476766